In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[]

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [5]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
X_train = dataset[0][0][0].values
y_train = dataset[0][0][1].values
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [7]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
y_train = get_one_hot(y_train, 2)
y_test = get_one_hot(y_test, 2)

In [8]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31, 2)
x test shape: (250, 31)
y test shape: (250, 31, 2)


In [9]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=510)

In [10]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=250)

In [11]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [12]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (10, 240, 31)
y test shape: (10, 31, 2)


In [13]:
# Reshaping X_train for efficient modelling
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

In [14]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (10, 240, 31)
y test shape: (10, 31, 2)


In [15]:
# expected input data shape: (batch_size, timesteps, data_dim)

# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=25, return_sequences=True, input_shape=(240, 31)))
# The output layer
regressor.add(LSTM(25))
regressor.add(Dense(62, activation='relu'))
regressor.add(Reshape((31,2)))
# regressor.add(softmax(axis = -1))
regressor.add(Lambda(lambda  x: softmax(x, axis = -1)))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Fitting to the training set
regressor.fit(X_train,y_train,epochs=50,batch_size=10)

Epoch 1/50
510/510 [==============================] - 9s 17ms/step - loss: 0.6934 - acc: 0.4992
Epoch 2/50
510/510 [==============================] - 8s 16ms/step - loss: 0.6919 - acc: 0.5162
Epoch 3/50
510/510 [==============================] - 8s 16ms/step - loss: 0.6907 - acc: 0.5279
Epoch 4/50
510/510 [==============================] - 8s 16ms/step - loss: 0.6894 - acc: 0.5352
Epoch 5/50
510/510 [==============================] - 8s 16ms/step - loss: 0.6879 - acc: 0.5382
Epoch 6/50
510/510 [==============================] - 8s 16ms/step - loss: 0.6864 - acc: 0.5435
Epoch 7/50
510/510 [==============================] - 9s 17ms/step - loss: 0.6848 - acc: 0.5430
Epoch 8/50
510/510 [==============================] - 9s 17ms/step - loss: 0.6833 - acc: 0.5493
Epoch 9/50
510/510 [==============================] - 9s 17ms/step - loss: 0.6816 - acc: 0.5517
Epoch 10/50
510/510 [==============================] - 9s 17ms/step - loss: 0.6803 - acc: 0.5534
Epoch 11/50
510/510 [==================

In [16]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [17]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int

In [18]:
label.shape

(10, 31, 2)

In [19]:
label[:, :, 1]

array([[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 1, 0, 1, 1, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 1, 0],
       [1,

In [20]:
y_test[:, :, 1]

array([[1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
        0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0.],
       [1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
        1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1.],
       [1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
        1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
        1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0.],
       [1., 1

In [21]:
y_test.size

620

In [22]:
(sum(y_test[:, :, 1] == label[:, :, 1])/(y_test.size/2)).sum()

0.5129032258064516

In [ ]:

regressor.save('../model/LSTM/my_model2.h5')  # creates a HDF5 file 'my_model.h5'
del regressor  # deletes the existing model

In [25]:
from keras.models import load_model
from keras.activations import softmax

# returns a compiled model
# identical to the previous one
regressor1 = load_model('../model/LSTM/my_model2.h5', custom_objects={"softmax": softmax})
predicted_load = regressor1.predict(X_test)
label_load = predicted_load > 0.5
label_load = label_load * 1 # Convert boolean to int

array([[[0.6681694 , 0.33183065],
        [0.67003244, 0.32996756],
        [0.47800538, 0.5219946 ],
        [0.45391676, 0.5460832 ],
        [0.5717928 , 0.42820722],
        [0.5       , 0.5       ],
        [0.5       , 0.5       ],
        [0.5742939 , 0.42570612],
        [0.47082087, 0.52917916],
        [0.5       , 0.5       ],
        [0.6377442 , 0.36225584],
        [0.3181387 , 0.6818613 ],
        [0.29697004, 0.70302993],
        [0.40600312, 0.5939969 ],
        [0.3952756 , 0.6047244 ],
        [0.5455349 , 0.4544651 ],
        [0.35796326, 0.64203674],
        [0.6971523 , 0.3028477 ],
        [0.4322768 , 0.5677232 ],
        [0.38664737, 0.61335266],
        [0.5       , 0.5       ],
        [0.69147503, 0.308525  ],
        [0.5       , 0.5       ],
        [0.5       , 0.5       ],
        [0.6872881 , 0.31271186],
        [0.2685327 , 0.73146737],
        [0.54715693, 0.45284313],
        [0.42825785, 0.5717421 ],
        [0.59079707, 0.40920293],
        [0.224